In [5]:
from langchain.chat_models import ChatOpenAI, ChatAnthropic
# 를 사용해서 open ai를 사용한다. ChatOpenAI를 사용해야 저렴하게 사용할 수 있음 3.5 turbo 사용.
# 그리고 langchain.chat_models를 가져올때 ChatOpenAI가 아닌 다른 모델도 가져올 수 있다.
# key만 있으면 사용할 수 있음

chat = ChatOpenAI()
# chat2 = ChatAnthropic() 다른 모델도 이렇게 사용할 수 있음

# chat = ChatOpenAI(
#     max_tokens=
#     temperature= 
#       ...
# )
# 위와 같은 설정으로 모델을 어떻게 사용할 것인지 정할 수 있다.

b = chat.predict("How many people in South Korea?")

b

'According to the latest data, the population of South Korea is approximately 51 million people.'

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# HumanMessage : 우리가 보내는 것. 사용자의 메세지
# AIMessage : AI에 의해 보내지는 것 
# SystemMessage : 우리가 LLM에 설정들을 제공하기 위한 Message. 프롬프트

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian."
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)
# 위와 같은 방식으로 더 복잡한 대화를 만들 수 있다.
# 일종의 설정을 AI에게 넘겨줬고, 마치 AI가 답변을 한 것으로 보여준다.
# 위의 결과값 :
# AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono qui per aiutarti con le tue domande di geografia. Avete altre domande?')


AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono qui per aiutarti con le tue domande di geografia. Avete altre domande?')

In [6]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate : template을 message로부터 만든다.
# PromptTemplate : 그냥 String을 이용해서 template을 만든다.

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)

prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,000 miles (14,500 kilometers) when measured in a straight line. However, the actual distance by air or sea travel may vary depending on the specific route taken.'

In [ ]:
# 위의 prompt messages를 사용해서 위의 방식을 조금 더 간편하게 바꿀 수 있다.
template = ChatPromptTemplate.from_messages([
    # SystemMessage(
    #     content="You are a geography expert. And you only reply in Italian."
    # ),
    # AIMessage(content="Ciao, mi chiamo Paolo!"),
    # HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)
# 결과값: AIMessage(content='Γεια σας, το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')


AIMessage(content='Γεια σας, το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [8]:
from langchain.schema import BaseOutputParser
# 결과값을 DB에 맞게 넣는다거나, distionary 또는 tuple에 넣을 형식으로 변형하기 위해서 사용

# BaseOutputParser를 확장
class CommaOutputParser(BaseOutputParser): 
    # parse라는 메소드가 반드시 필요하다.
    def parse(self, text):
        # return text.strip().split(",") # strip은 trim과 같음(앞뒤 공백 없앤다)
        # 위의 내용을 조금 더 발전시켜서
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how,  are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. DO NOT reply with anything else."
        ),
        ("human", "{question}")
    ]
)

prompt = template.format_messages(max_items=10, question="What are the colors?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)
# 결과값
# ['red',
#  'blue',
#  'green',
#  'yellow',
#  'orange',
#  'purple',
#  'pink',
#  'black',
#  'white',
#  'brown']

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
# LCEL : LangChain Expression Language
# chain 생성하기 : 템플릿 | chat model | parser
chain = template | chat | CommaOutputParser()

# chain 실행방법
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})
# 결과값 : ['squirtle', 'bulbasaur', 'charmander', 'pikachu', 'jigglypuff']

# LangChain이 내부적으로
# 1. template.format_messages 호출
# 2. chat.predict 호출
# 3. parse 호출


['squirtle', 'bulbasaur', 'charmander', 'pikachu', 'jigglypuff']